<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA4_%ED%8C%80%ED%88%AC%EC%88%98_%EC%84%B8%EC%9D%B4%EB%B2%84_%EB%A7%A4%ED%8A%B8%EB%A6%AD%EC%8A%A4_%ED%95%A9%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
def load_Data(name):
    years = ['2016','2017','2018','2019','2020']
    name_salt = '2020빅콘테스트_스포츠투아이_제공데이터_'
    PATH = path
    df = pd.DataFrame()
    for year in years:
        if year == '2016':
            df = pd.read_csv(PATH + name_salt + name+'_'+year+'.csv', encoding='CP949')
        else :
            new_df = pd.read_csv(PATH + name_salt + name+'_'+year+'.csv', encoding='CP949')
            df = pd.concat([df, new_df])
    return df

In [4]:
team_pitcher = load_Data('팀투수')

In [ ]:
team_pitcher

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,BF,PA,AB,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,WP,BK,ERR,R,ER,P_WHIP_RT,P2_WHIP_RT,CB_WHIP_RT
0,20160401HHLG0,20160401,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000
1,20160401HHLG0,20160401,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000
2,20160401HTNC0,20160401,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857
3,20160401HTNC0,20160401,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000
4,20160401KTSK0,20160401,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,20200719,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000
636,20200719OBHT0,20200719,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000
637,20200719OBHT0,20200719,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000
638,20200719WOSK0,20200719,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857


In [ ]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP"
new_columns = new_columns.split(',')

In [ ]:
team_pitcher.columns = new_columns

In [ ]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020
636,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020
637,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020
638,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020


In [ ]:
team_pitcher['일자'] = pd.to_datetime(team_pitcher['일자'],format='%Y%m%d', errors='raise')
team_pitcher['시즌'] = team_pitcher['일자'].dt.year 

 평균자책점(ERA) 구하기 **팀의 수비력이나 우연에 의해 영향을 받는 면이 존재

In [ ]:
def createERA(row):
    previous = team_pitcher[team_pitcher['팀코드'] == row['팀코드']].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['평균자책점'] = 0
      return row
    up = previous['자책점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['평균자책점']
        return row
    row['평균자책점'] = (up * 9) / down
    return row
team_pitcher = team_pitcher.apply(createERA, axis=1)

In [ ]:
team_pitcher.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점']

In [ ]:
def createRA(row):
    previous = team_pitcher[team_pitcher['팀코드'] == row['팀코드']].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['RA/9'] = 0
      return row
    up = previous['실점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['RA/9']
        return row
    row['RA/9'] = (up * 9) / down
    return row
team_pitcher = team_pitcher.apply(createRA, axis=1)

In [ ]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,4.817905,5.261876
636,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.292394,4.746883
637,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,4.999407,5.576261
638,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,4.874109,5.419240


*ERC* => 평균자책점에서 투수의 능력 이외의 것을 배제 시킨 것

In [ ]:
def createERC(row):
    previous = team_pitcher[team_pitcher['팀코드'] == row['팀코드']].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['ERC'] = 0
      return row
    previous_sum = previous.sum()
    PTB = 0.89 * (1.255 * (previous_sum['안타'] - previous_sum['홈런']) + 4 * previous_sum['홈런']) + 0.475 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4'])
    ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / (previous_sum['타자'] * previous_sum['이닝*3'] / 3) ) - 0.56
    if ERC < 2.24:
      ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / (previous_sum['타자'] * previous_sum['이닝*3'] / 3) ) * 0.75
    row['ERC'] = ERC
    return row

In [ ]:
team_pitcher = team_pitcher.apply(createERC, axis=1)

FIP: 수비무관자책점 **논쟁이 좀 많음 "FIP가 갖는 의미가 큰 이유는 기존의 평균자책점(ERA) 시스템이 갖는 문제점인 '투수의 실점억제력과 수비로 인한 실점억제력을 분리시키지 못함으로써 실점에 있어서 투수의 원래 책임보다 많은 부분을 투수에 지우는 것’에 대안을 제시했다는 점이다." 하지만 분명히 투수도 1) 뜬공과 땅볼의 비율 2) 구위 3) 제구에 의해 BABIP에 영향을 끼칠 수 있으며 주자의 견제능력, 투수본인의 수비능력, 셋 포지션에서의 구위하락 등 투수가 실점을 억제할 수 있는 요소들 중 일부는 FIP에 반영되지 못하고 있다.

In [ ]:
C_dic = {}
for year in range(2016,2021):
  team_pitcher_year = team_pitcher[team_pitcher['시즌'] == year].copy()
  team_pitcher_year = team_pitcher_year.sum() 
  C = (( 9 * team_pitcher_year['자책점'] - ( 13 * team_pitcher_year['홈런'] + 3 * (team_pitcher_year['4구'] + team_pitcher_year['사구'] - team_pitcher_year['고4']) - 2 * team_pitcher_year['삼진'])) / (team_pitcher_year['이닝*3'] / 3))
  C_dic[year] = C

In [ ]:
C_dic

{2016: 3.7732111715557406,
 2017: 3.7201314451138408,
 2018: 3.810512298292339,
 2019: 3.3880686560934894,
 2020: 3.596723044397463}

In [ ]:
def createFIP(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['FIP'] = 0
        return row
    previous_sum = previous.sum()
    FIP = (( 13 * previous_sum['홈런'] + 3 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4']) - 2 * previous_sum['삼진']) / (previous_sum['이닝*3'] / 3)) + C_dic[row['시즌']]
    row['FIP'] = FIP
    return row

In [ ]:
team_pitcher = team_pitcher.apply(createFIP, axis=1)

In [ ]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9,ERC,FIP
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,4.817905,5.261876,4.409593,4.923154
636,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.292394,4.746883,3.970355,4.189616
637,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,4.999407,5.576261,4.741899,4.415714
638,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,4.874109,5.419240,4.804711,4.959550


In [ ]:
team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA4-팀투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')

LOB% =>  잔루처리율 대부분 70 ~ 72로 평군회귀 "일반적으로 대부분의 투수들은 장기적인 관점에서 보았을 때 리그평균에 가까운 수치(70~72%)를 기록하게 된다. 따라서 평균에서 벗어난 투수들은 미래에 평균으로 회귀하게 될 것이라고 예측할 수 있다. 예를들어 단기적으로 LOB%이 낮은 선수들은 ERA가 상승하게 되지만 LOB%이 평균에 가깝게 회귀하면서 미래에는 ERA가 지금 수준에 비해 낮아질 것이라는 해석이 가능하며, 반대로 LOB%이 높은 선수는 LOB%이 평균에 가깝게 회귀하면서 미래에는 지금 수준에 비해 ERA가 높을 것이라는 해석이 가능하다." => ERA 예측에 도움

In [ ]:
def createLOB(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['LOB'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - previous_sum['실점']
    down = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - (1.4 * previous_sum['홈런'])
    if down == 0:
      row['LOB'] = 0
      return row 
    row['LOB'] = up / down
    return row

In [ ]:
team_pitcher = team_pitcher.apply(createLOB, axis=1)

홈런율, 삼진율 => 타석당 홈련, 삼진 비율

In [ ]:
def createK_HR(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    previous_sum = previous.sum()
    up_HR = previous_sum['홈런'] 
    up_k = previous_sum['삼진']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    row['홈런율'] = up_HR / down
    row['삼진율'] = up_k / down
    return row

In [ ]:
team_pitcher = team_pitcher.apply(createK_HR, axis=1)

K/9 BB/9 HR/9 K/BB

In [ ]:
def create9(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    previous_sum = previous.sum()
    up_K = previous_sum['삼진'] 
    up_BB = previous_sum['4구'] - previous_sum['고4']
    up_HR = previous_sum['홈런']
    down = previous_sum['이닝*3'] / 3
    if down == 0 or up_BB == 0 :
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    row['K9'] = up_K / down * 9
    row['BB9'] = up_BB / down * 9
    row['HR9'] = up_HR / down * 9
    row['KBB'] = up_K / up_BB
    return row

In [ ]:
team_pitcher = team_pitcher.apply(create9, axis=1)

In [ ]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9,ERC,FIP,LOB,홈런율,삼진율,K9,BB9,HR9,KBB
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,4.817905,5.261876,4.409593,4.923154,0.679640,0.030008,0.174134,6.774665,3.091352,1.167479,2.191489
636,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.292394,4.746883,3.970355,4.189616,0.684932,0.020524,0.199127,7.675810,3.114090,0.791147,2.464865
637,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,4.999407,5.576261,4.741899,4.415714,0.654102,0.019154,0.166800,6.697923,3.188724,0.769139,2.100503
638,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,4.874109,5.419240,4.804711,4.959550,0.683666,0.023089,0.176354,7.102732,4.248812,0.929929,1.671698


In [ ]:
team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA4-팀투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')

In [5]:
team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA4-팀투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')

In [10]:
def createWhip(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['whip'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구']
    down = previous_sum['이닝*3'] / 3
    if down == 0:
      row['whip'] = 0
      return row 
    row['whip'] = up / down
    return row

In [11]:
team_pitcher = team_pitcher.apply(createWhip, axis=1)

In [12]:
team_pitcher

,Unnamed: 0,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9,ERC,FIP,LOB,홈런율,삼진율,K9,BB9,HR9,KBB,whip
0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,4.817905,5.261876,4.409593,4.923154,0.679640,0.030008,0.174134,6.774665,3.091352,1.167479,2.191489,1.401340
6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.292394,4.746883,3.970355,4.189616,0.684932,0.020524,0.199127,7.675810,3.114090,0.791147,2.464865,1.367207
6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,4.999407,5.576261,4.741899,4.415714,0.654102,0.019154,0.166800,6.697923,3.188724,0.769139,2.100503,1.520475
6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,4.874109,5.419240,4.804711,4.959550,0.683666,0.023089,0.176354,7.102732,4.248812,0.929929,1.671698,1.530285


In [13]:
team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA4-팀투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)